### Import packages:

In [29]:
import pandas as pd
from datetime import datetime
import os

### Define functions:

In [30]:
def CalculatePunctuality(schedule, confirmed):
    schedule_mins = schedule.str[:2].astype('int32') * 60 + schedule.str[-2:].astype('int32')
    confirmed_mins = confirmed.str[:2].astype('int32') * 60 + confirmed.str[-2:].astype('int32')
    return confirmed_mins - schedule_mins

def CollectDataFrames(scenario):
    
    scenario = scenario.lower()
    if scenario not in ['actual', 'plan']:
        return []
    
    dir = os.listdir(f"FlightData\\{scenario.title()}")
    filenames = [file for file in dir if file != f"flights_departures_{scenario}.csv"]

    current_date = datetime.today().strftime('%Y-%m-%d')
    cutoff_date = ""
    dfs = []

    for filename in filenames:
        df_tmp = pd.read_csv(os.path.join("FlightData", scenario.title(), filename))
        dfs.append(df_tmp[df_tmp["Date"] > cutoff_date])
        cutoff_date = df_tmp["Date"].max()
    
    return dfs

### Actual-scenario:

In [31]:
data_frames = CollectDataFrames("actual")
    
df_actual = pd.concat(data_frames, sort=False)
df_actual = df_actual[df_actual["Confirmed"].str[0] <= "2"]
df_actual["CreationDate"] = datetime.today().strftime('%Y-%m-%d')
df_actual["PunctualityMins"] = CalculatePunctuality(df_actual["Scheduled"], df_actual["Confirmed"])
df_actual["Punctuality"] = df_actual.apply(lambda row : 1 if row["PunctualityMins"] > 0 else -1, axis=1)
df_actual.to_csv(os.path.join("FlightData","Actual", "flights_departures_actual.csv"), index=False)

### Plan-scenario:

In [32]:
data_frames = CollectDataFrames("plan")
    
df_plan = pd.concat(data_frames, sort=False)
df_plan["CreationDate"] = datetime.today().strftime('%Y-%m-%d')
df_plan["PunctualityMins"] = 0
df_plan["Punctuality"] = 0
df_plan.to_csv(os.path.join("FlightData", "Plan", "flights_departures_plan.csv"), index=False)

### Combine Scenarios & Add Time-related columns:

In [33]:
df = pd.concat([df_actual, df_plan])
df = df.reset_index(drop=True)

df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.week
df['Day'] = df['Date'].dt.day
df['WeekDay'] = df['Date'].dt.day_name()

df["IsMorning"] = True
i = df[(df.Scheduled.str[0:2].astype('int32') >= 12)]["IsMorning"].index
df.loc[i,"IsMorning"] = False

### Save Datasets:

In [35]:
df_actual = df[df.Scenario == "Actual"].reset_index(drop=True)
df_plan = df[df.Scenario == "Plan"].reset_index(drop=True)

df.to_pickle(os.path.join("FlightData", "flight_departures.pickle"))
df_actual.to_pickle(os.path.join("FlightData", "flight_departures_actual.pickle"))
df_plan.to_pickle(os.path.join("FlightData", "flight_departures_plan.pickle"))